In [1]:
%matplotlib inline
from matplotlib  import pyplot as plt
import numpy as np
import pandas as pd
from collections import defaultdict, namedtuple
import networkx as nx
from tqdm import tqdm

In [2]:
def split2int(s):
    return list(map(int, s.split()))

In [3]:
Req = namedtuple('Request', ['v', 'e', 'n'])

In [4]:
# data = 'me_at_the_zoo.in'
data = 'trending_today.in'
# data = 'videos_worth_spreading.in'
# data = 'kittens.in'
outpath = 'outputs/{}.out'.format(data.split('.')[0])
reqs = {}
latency = {}
v2reqs = defaultdict(list)
c2ends = defaultdict(set)

CENTER = -1
with open('data/{}'.format(data), 'r') as f:
    V, E, R, C, X = split2int(f.readline())
    v2size = {i: int(size)
              for i, size in enumerate(f.readline().strip().split())}
    for endpoint_i in range(E):
        latency2center, n_caches = split2int(f.readline())
        latency[(endpoint_i, CENTER)] = latency2center                    
        # g.add_edge(end(endpoint_i), CENTER, d=latency2center)
        for _ in range(n_caches):
            cache_i, l = split2int(f.readline())
            latency[(endpoint_i, cache_i)] = l            
            c2ends[cache_i].add(endpoint_i)
        endpoint_i += 1
        
    for l in f:
        vid, eid, n = split2int(l)
        v2reqs[vid].append(Req(vid, eid, n))
        

In [5]:
vc2reqs = {}  # video, cache to list of requests
for v, reqs in tqdm(v2reqs.items()):
    for c in range(C):
        vc2reqs[(v, c)] = set([r for r in reqs if r.e in c2ends[c]])
    

100%|██████████| 10000/10000 [00:07<00:00, 1276.32it/s]


In [ ]:
I = {}
for c in tqdm(range(C)):
    for v in range(V):
        I[(c, v)] = sum(r.n * (latency[(r.e, CENTER)] - latency[(r.e, c)] ) for r in vc2reqs[(v, c)])


100%|██████████| 100/100 [00:12<00:00,  7.98it/s]


In [ ]:
caps = {i: X for i in range(C)}
c2v = defaultdict(set)
req2c = {}  # which cache serves *the* request
candidate_cv_tuples = sorted(I, key=I.__getitem__, reverse=True)
while True:
    if len(candidate_cv_tuples) % 10 == 0:
        print('remaing {} tuples'.format(len(candidate_cv_tuples)))
    candidate_cv_tuples = sorted(candidate_cv_tuples, key=I.__getitem__, reverse=True)
    success = False
    while True:
        cb, vb = candidate_cv_tuples.pop(0)
        if caps[cb] >= vb:
            caps[cb] -= v2size[vb]
            c2v[cb].add(vb)
            success = True
            break

    if not success:
        # not enough capacity
        # we are done
        break
        
    # udpate which cache serves each request
    for r in v2reqs[vb]:
        if r in req2c:
            if latency[(r.e, req2c[r])] > latency[(r.e, cb)]:
                req2c[r] = cb
        else:
            req2c[r] = cb
            
    # update the I matrix
    for c in range(C):
        improvement = 0
        if c != cb:
            for r in vc2reqs[(vb, c)]:
                if r in req2c:  # already served
                    if latency[(r.e, req2c[r])] > latency[(r.e, c)]:
                        improvement += r.n * (latency[(r.e, req2c[r])] - latency[(r.e, c)])
                else:
                    improvement += r.n * (latency[(r.e, CENTER)] - latency[(r.e, c)])
            I[(c, vb)] = improvement
            
            

remaing 1000000 tuples
remaing 999990 tuples
remaing 999980 tuples
remaing 999970 tuples
remaing 999960 tuples
remaing 999950 tuples
remaing 999940 tuples
remaing 999930 tuples
remaing 999920 tuples
remaing 999910 tuples
remaing 999900 tuples
remaing 999890 tuples
remaing 999880 tuples
remaing 999870 tuples
remaing 999860 tuples
remaing 999850 tuples
remaing 999840 tuples
remaing 999830 tuples
remaing 999820 tuples
remaing 999810 tuples
remaing 999800 tuples
remaing 999790 tuples
remaing 999780 tuples
remaing 999770 tuples
remaing 999760 tuples
remaing 999750 tuples
remaing 999740 tuples
remaing 999730 tuples
remaing 999720 tuples
remaing 999710 tuples
remaing 999700 tuples
remaing 999690 tuples
remaing 999680 tuples
remaing 999670 tuples
remaing 999660 tuples
remaing 999650 tuples
remaing 999640 tuples
remaing 999630 tuples
remaing 999620 tuples
remaing 999610 tuples
remaing 999600 tuples
remaing 999590 tuples
remaing 999580 tuples
remaing 999570 tuples
remaing 999560 tuples
remaing 9

In [ ]:
# sort by number of requests (big is good) and size (small is good)
reqs.sort_values(by=['n', 'vsize'], ascending=[False, True], inplace=True)

In [ ]:
reqs_array = reqs.as_matrix().tolist()

In [ ]:
cache_servers = list(filter(lambda n: n.startswith('c'), g.nodes()))

In [ ]:
cache2videos = defaultdict(set)

In [ ]:
for n in cache_servers:
    g.node[n]['remain'] = X

# select the request with max number and min size    
ii = 0
while len(reqs_array) > 0:
    ii += 1
    if ii % 10000 == 0:
        print("{} / {}".format(ii, R))
    v, e, n, s = reqs_array.pop(0)
    # if video is already in e, ignore???
    # assign it to the fastest cache server
    able_cache_servers = list(filter(lambda c: g.node[c]['remain'] >= s,
                                     g.neighbors(end(e))))
    if len(able_cache_servers) == 0:  # cache server cannot hold        
        # cache2videos[CENTER].add(v)
        pass
    else:
        best_c = max(able_cache_servers, key=lambda c: end2cache_latency[e][int(c[1:])] - g[end(e)][c]['d'])
        cache2videos[int(best_c[1:])].add(v)
        g.node[best_c]['remain'] -= s    

In [ ]:
with open(outpath, 'w') as f:
    f.write('{}\n'.format(len(cache2videos)))
    for c, videos in cache2videos.items():
        f.write('{} {}\n'.format(c, ' '.join(map(str, videos))))

In [ ]:
vids_by_count = reqs.groupby('vid')['n'].sum().sort_values(ascending=False)

In [ ]:
cache_servers = list(filter(lambda n: n.startswith('c'), g.nodes()))
for n in cache_servers:
    g.node[n]['remain'] = X

In [ ]:
req_by_videos = {}
for r, sub_df in reqs.groupby('vid'):
    req_by_videos[r] = sub_df

In [ ]:
req_by_videos = sorted(req_by_videos.items(), key=lambda t: t[1]['n'].sum(), reverse=True)
req_by_videos = [(r, sdf.as_matrix()) for r, sdf in req_by_videos]

In [ ]:
cache2videos = defaultdict(set)
for n in cache_servers:
    g.node[n]['remain'] = X
    
e2c_best = {e: CENTER for e in range(E)}

for v, rs in tqdm(req_by_videos):
    # endpoints that need video i
    # put v into cache that get the max gain
    gain_old = 0
    for i in range(rs.shape[0]):        
        _, e, n, _ = rs[i]
        best_cache = e2c_best[e]
        gain_old += n * (end2cache_latency[e][CENTER] - end2cache_latency[e][best_cache])
    print('gain_old', gain_old)
    
    capable_caches = [c for c in range(C) if g.node[cache(c)]['remain'] >= video_sizes[v]]
    if len(capable_caches) == 0:
        continue
        
    best_gain = gain_old
    best_c = None
    for c in capable_caches:
        gain_new = 0
        for i in range(rs.shape[0]):        
            _, e, n, _ = rs[i]
            if c in end2cache_latency[e]:
                gain_new += n * (end2cache_latency[e][CENTER] - end2cache_latency[e][c])
        if best_gain < gain_new:
            best_gain = gain_new
            best_c = c
        print('gain_new', gain_new)
    # print(best_c)
    if best_c is not None:
        g.node[cache(best_c)]['remain'] -= video_sizes[v]
    #     best_c = int(best_c[1:])
        cache2videos[best_c].add(v)

        for e, c in e2c_best.items():
            if best_c in end2cache_latency[e]:
                if end2cache_latency[e][c] > end2cache_latency[e][best_c]:
                    e2c_best[e] = best_c

In [ ]:
with open(outpath, 'w') as f:
    f.write('{}\n'.format(len(cache2videos)))
    for c, videos in cache2videos.items():
        f.write('{} {}\n'.format(c, ' '.join(map(str, videos))))

In [ ]:
cache2videos